# VacationPy
___

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Load cities.csv to a DataFrame

cities_data = pd.read_csv("../output_data/cities.csv")
#cities_data.set_index("City_ID")

# Make sure to handle NaN values

cities_data = cities_data.dropna()
cities_data.set_index("City_ID")
cities_data

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key

gmaps.configure(api_key = g_key)

In [4]:
# Convert 'Lat' and 'Lng' and Humidity to float and Store 

locations = cities_data[["Latitude", "Longitude"]].astype(float)

Humidities = cities_data["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer

fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = Humidities, 
                                 dissipating = False, 
                                 opacity = 0.8,
                                 max_intensity=200,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = cities_data.loc[
                                    (cities_data["Max Temp"] < 85) & 
                                    (cities_data["Max Temp"] > 65) & 
                                    (cities_data["Wind Speed"] < 10) & 
                                    (cities_data["Cloudiness"] < 10)
                                  ].dropna()
narrowed_city_df.set_index("City_ID")

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
City_ID,,,,,,,,
29,Najran,SA,17.49,44.13,71.60,35,5,4.38
40,Umm lajj,SA,25.02,37.27,76.10,39,7,5.84
105,Pecos,US,31.42,-103.49,79.00,9,1,3.36
141,Saint-philippe,RE,-21.36,55.77,77.00,83,3,6.93
192,Saint-pierre,RE,-21.34,55.48,77.00,83,8,6.93
206,Saint george,US,37.10,-113.58,72.00,26,1,5.82
222,Bataipora,BR,-22.30,-53.27,75.54,66,7,4.09
224,Amarpatan,IN,24.32,80.98,75.36,23,0,3.06
235,Lom sak,TH,16.78,101.24,82.00,64,5,1.23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates

hotel_df = narrowed_city_df[["City", "Country", "Max Temp", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Latitude,Longitude,Hotel Name
29,Najran,SA,71.60,17.49,44.13,
40,Umm lajj,SA,76.10,25.02,37.27,
105,Pecos,US,79.00,31.42,-103.49,
141,Saint-philippe,RE,77.00,-21.36,55.77,
192,Saint-pierre,RE,77.00,-21.34,55.48,
206,Saint george,US,72.00,37.10,-113.58,
222,Bataipora,BR,75.54,-22.30,-53.27,
224,Amarpatan,IN,75.36,24.32,80.98,
235,Lom sak,TH,82.00,16.78,101.24,
240,Howard springs,AU,84.00,-12.50,131.05,


In [8]:
# Build the base URL for google apis
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for a hotel

params = { "radius": 5000,
           "types": "lodging",
           "key": g_key
         }

# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():
    
    # get latitude , longitude from df
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    response = requests.get(base_url, params=params)
    
    # Convert to json
    response_json = response.json()
    
    # Grab the first hotel from the results and store the name
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Max Temp,Latitude,Longitude,Hotel Name
29,Najran,SA,71.60,17.49,44.13,محمد هادي ال فطيح
40,Umm lajj,SA,76.10,25.02,37.27,Tanal Hotel Apartment
105,Pecos,US,79.00,31.42,-103.49,Knights Inn Pecos
141,Saint-philippe,RE,77.00,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
192,Saint-pierre,RE,77.00,-21.34,55.48,Lindsey Hôtel
206,Saint george,US,72.00,37.10,-113.58,Best Western Plus Abbey Inn
222,Bataipora,BR,75.54,-22.30,-53.27,Hotel Vale do Ivinhema
224,Amarpatan,IN,75.36,24.32,80.98,कब्रिस्तान
235,Lom sak,TH,82.00,16.78,101.24,บ้านวรฉัตร หล่มสัก
240,Howard springs,AU,84.00,-12.50,131.05,buffalo beach hostel


In [9]:
# Create list of hotels to use for hover_text

hotels = []

for hotel in hotel_df["Hotel Name"]:
    hotels.append(hotel)

print(hotels)

['محمد هادي ال فطيح', 'Tanal Hotel Apartment', 'Knights Inn Pecos', 'Chambres d\'hôte "La Trinité"', 'Lindsey Hôtel', 'Best Western Plus Abbey Inn', 'Hotel Vale do Ivinhema', 'कब्रिस्तान', 'บ้านวรฉัตร หล่มสัก', 'buffalo beach hostel', 'Hotel Kununurra', 'ibis Styles Mt Isa Verona', 'Inn On Destin Harbor', 'Hotel Kanyapur', '', 'Hôtel Telwa Bungalow', 'MHL Turismo SA', 'Hotel Dawei', 'Red Sea Hotel', 'Sapphire City Motor Inn', 'La Quinta Inn & Suites by Wyndham Buena Park', 'Tinidee Hotel@Ranong', 'Ezdan Hotel', 'Hôtel Trois Étoiles de Biltine', 'Hotel Lemurian Heritage', 'Villavela Hotel Boutique', 'Kimia IV Hotel', 'D-26/2, Dolphin Hill,', 'Hotel Avenida', 'Pine Tree Motel', 'Hotel da Cidade', 'Aditya hotel and restaurant']


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info, hover_text = hotels)
fig.add_layer(marker_layer)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))